In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
from six.moves import range
from sklearn.metrics import confusion_matrix
import tensorflow as tf
print(tf.__version__)

%matplotlib inline

1.3.0


In [2]:
train_folders = ['../unit-1/notMNIST_large/A', '../unit-1/notMNIST_large/B', '../unit-1/notMNIST_large/C', 
                 '../unit-1/notMNIST_large/D', '../unit-1/notMNIST_large/E', '../unit-1/notMNIST_large/F', 
                 '../unit-1/notMNIST_large/G', '../unit-1/notMNIST_large/H', '../unit-1/notMNIST_large/I', '../unit-1/notMNIST_large/J']
test_folders = ['../unit-1/notMNIST_small/A', '../unit-1/notMNIST_small/B', '../unit-1/notMNIST_small/C', 
                '../unit-1/notMNIST_small/D', '../unit-1/notMNIST_small/E', '../unit-1/notMNIST_small/F', 
                '../unit-1/notMNIST_small/G', '../unit-1/notMNIST_small/H', '../unit-1/notMNIST_small/I', '../unit-1/notMNIST_small/J']

In [3]:
train_datasets = []
for folder in train_folders:
  set_filename = folder + '.pickle'
  train_datasets.append(set_filename)
    
test_datasets = []
for folder in test_folders:
  set_filename = folder + '.pickle'
  test_datasets.append(set_filename)

In [4]:
train_dataset = []
num=0
for file in train_datasets:
  f = open(file,"rb")
  dataset = pickle.load(f)
  num = num +1
  train_dataset.append(dataset[0:10000,:,:])
  print(train_dataset[num-1].shape)

(10000, 28, 28)
(10000, 28, 28)
(10000, 28, 28)
(10000, 28, 28)
(10000, 28, 28)
(10000, 28, 28)
(10000, 28, 28)
(10000, 28, 28)
(10000, 28, 28)
(10000, 28, 28)


In [5]:
test_dataset = []
num = 0
for file in test_datasets:
  f = open(file,"rb")
  dataset = pickle.load(f)
  num = num +1
  test_dataset.append(dataset)
  print(dataset.shape)


## split train dataset to get validation set
train_labels = []
test_labels = []
for label in range(len(train_datasets)):
  for i in range(10000):
    train_labels.append(label) 
    
for label in range(len(test_datasets)):
  for i in range(test_dataset[label].shape[0]):
    test_labels.append(label) 
    

(1872, 28, 28)
(1873, 28, 28)
(1873, 28, 28)
(1873, 28, 28)
(1873, 28, 28)
(1872, 28, 28)
(1872, 28, 28)
(1872, 28, 28)
(1872, 28, 28)
(1872, 28, 28)


In [6]:
temp = []
for i in range(10):
  for j in range(train_dataset[i].shape[0]):
    temp.append(train_dataset[i][j])

train_dataset, temp = temp, train_dataset

temp = []
for i in range(10):
  for j in range(test_dataset[i].shape[0]):
    temp.append(test_dataset[i][j])

test_dataset, temp = temp, test_dataset

test_dataset = np.array(test_dataset)
test_dataset.shape

(18724, 28, 28)

In [7]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
train_dataset = np.array(train_dataset)
test_dataset = np.array(test_dataset)

In [8]:
train_dataset = train_dataset.reshape((-1,784))
test_dataset = test_dataset.reshape((-1,784))

In [9]:
trainData, validData, trainLabels, validLabels = train_test_split(train_dataset, train_labels, test_size = 0.1)

In [10]:
## function for one-hot encoding
num_labels = 10
def reformat(labels):
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return labels
trainLabels = reformat(trainLabels)
validLabels = reformat(validLabels)
test_labels = reformat(test_labels)

In [11]:
## final check of shape of dataset
print(trainData.shape, trainLabels.shape)
print(validData.shape, validLabels.shape)
print(test_dataset.shape, test_labels.shape)

(90000, 784) (90000, 10)
(10000, 784) (10000, 10)
(18724, 784) (18724, 10)


In [14]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 1000
image_size = 28
beta =0.01

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(trainData[:train_subset, :])
  tf_train_labels = tf.constant(trainLabels[:train_subset])
  tf_valid_dataset = tf.constant(validData)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  ### adding regularizer
  regularizer = tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(loss + beta * regularizer)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [15]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, trainLabels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), validLabels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 49.868832
Training accuracy: 5.9%
Validation accuracy: 9.2%
Loss at step 100: 11.391643
Training accuracy: 81.4%
Validation accuracy: 68.9%
Loss at step 200: 4.253584
Training accuracy: 92.0%
Validation accuracy: 73.6%
Loss at step 300: 1.793914
Training accuracy: 95.2%
Validation accuracy: 76.4%
Loss at step 400: 0.927163
Training accuracy: 96.0%
Validation accuracy: 78.2%
Loss at step 500: 0.619005
Training accuracy: 96.6%
Validation accuracy: 78.9%
Loss at step 600: 0.508621
Training accuracy: 96.7%
Validation accuracy: 79.1%
Loss at step 700: 0.468776
Training accuracy: 96.7%
Validation accuracy: 79.1%
Loss at step 800: 0.454259
Training accuracy: 96.7%
Validation accuracy: 79.1%
Test accuracy: 86.1%


In [16]:
## adding a hidden layer to SGD implementation

batch_size = 128
h1_size = 1024
beta = 0.01

def layerfunc(x,weights,biases):
  layer_1 = tf.add(tf.matmul(x,weights['h1']),biases['h1'])
  layer_1 = tf.nn.relu(layer_1)
  logits = tf.matmul(layer_1, weights['out']) + biases['out']
  return logits

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(validData)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  #weights = tf.Variable(
  #  tf.truncated_normal([image_size * image_size, num_labels]))
  #biases = tf.Variable(tf.zeros([num_labels]))
  weights = {
    'h1' : tf.Variable(tf.truncated_normal([image_size * image_size, h1_size])),
    'out' : tf.Variable(tf.truncated_normal([h1_size,num_labels]))
  }
  biases = {
    'h1' : tf.Variable(tf.zeros([h1_size])),
    'out' : tf.Variable(tf.zeros([num_labels]))
  }
  
  # Training computation.
  #logits = tf.matmul(tf_train_dataset, weights) + biases
  logits = layerfunc(tf_train_dataset,weights,biases)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  regularizers = tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(weights['out'])
  loss = tf.reduce_mean(loss + beta*regularizers)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(layerfunc(tf_valid_dataset,weights,biases))
  test_prediction = tf.nn.softmax(layerfunc(tf_test_dataset,weights,biases))

In [17]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (trainLabels2.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = trainData[offset:(offset + batch_size), :]
    batch_labels = trainLabels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), validLabels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3506.409668
Minibatch accuracy: 7.8%
Validation accuracy: 22.3%
Minibatch loss at step 500: 21.183311
Minibatch accuracy: 86.7%
Validation accuracy: 84.1%
Minibatch loss at step 1000: 0.898021
Minibatch accuracy: 82.0%
Validation accuracy: 83.5%
Minibatch loss at step 1500: 0.696875
Minibatch accuracy: 86.7%
Validation accuracy: 83.2%
Minibatch loss at step 2000: 0.663497
Minibatch accuracy: 87.5%
Validation accuracy: 83.5%
Minibatch loss at step 2500: 0.662398
Minibatch accuracy: 88.3%
Validation accuracy: 83.9%
Minibatch loss at step 3000: 0.705031
Minibatch accuracy: 83.6%
Validation accuracy: 84.0%
Test accuracy: 90.2%


In [18]:
### to verify overfitting by reducing the number of samples

num_steps = 3001
trainData2 = trainData[:500,:]
trainLabels2 = trainLabels[:500]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (trainLabels2.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = trainData2[offset:(offset + batch_size), :]
    batch_labels = trainLabels2[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), validLabels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3462.482910
Minibatch accuracy: 11.7%
Validation accuracy: 25.2%
Minibatch loss at step 500: 21.053051
Minibatch accuracy: 100.0%
Validation accuracy: 77.2%
Minibatch loss at step 1000: 0.498861
Minibatch accuracy: 100.0%
Validation accuracy: 77.7%
Minibatch loss at step 1500: 0.303629
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 2000: 0.281876
Minibatch accuracy: 100.0%
Validation accuracy: 77.9%
Minibatch loss at step 2500: 0.267494
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at step 3000: 0.265417
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Test accuracy: 85.1%


In [22]:
## adding a hidden layer to SGD implementation
## adding dropout layer to reduce overfitting

batch_size = 128
h1_size = 1024
beta = 0.01

def layerfunc(x,weights,biases):
  layer_1 = tf.add(tf.matmul(x,weights['h1']),biases['h1'])
  layer_1 = tf.nn.relu(layer_1)
  logits = tf.matmul(layer_1, weights['out']) + biases['out']
  return logits

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(validData)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  #weights = tf.Variable(
  #  tf.truncated_normal([image_size * image_size, num_labels]))
  #biases = tf.Variable(tf.zeros([num_labels]))
  weights = {
    'h1' : tf.Variable(tf.truncated_normal([image_size * image_size, h1_size])),
    'out' : tf.Variable(tf.truncated_normal([h1_size,num_labels]))
  }
  biases = {
    'h1' : tf.Variable(tf.zeros([h1_size])),
    'out' : tf.Variable(tf.zeros([num_labels]))
  }
  
  # Training computation.
  #logits = tf.matmul(tf_train_dataset, weights) + biases
  layer_1 = tf.add(tf.matmul(tf_train_dataset,weights['h1']),biases['h1'])
  layer_1 = tf.nn.relu(layer_1)
  keep_prob = tf.placeholder("float")
  layer_1 = tf.nn.dropout(layer_1,keep_prob)
  logits = tf.matmul(layer_1, weights['out']) + biases['out']
  
  ## calculate loss
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  regularizers = tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(weights['out'])
  loss = tf.reduce_mean(loss + beta*regularizers)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(layerfunc(tf_valid_dataset,weights,biases))
  test_prediction = tf.nn.softmax(layerfunc(tf_test_dataset,weights,biases))

In [24]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (trainLabels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = trainData[offset:(offset + batch_size), :]
    batch_labels = trainLabels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), validLabels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3687.726562
Minibatch accuracy: 10.2%
Validation accuracy: 27.3%
Minibatch loss at step 500: 21.311592
Minibatch accuracy: 86.7%
Validation accuracy: 84.0%
Minibatch loss at step 1000: 1.010349
Minibatch accuracy: 82.8%
Validation accuracy: 83.2%
Minibatch loss at step 1500: 0.756688
Minibatch accuracy: 89.1%
Validation accuracy: 83.0%
Minibatch loss at step 2000: 0.739275
Minibatch accuracy: 84.4%
Validation accuracy: 83.3%
Minibatch loss at step 2500: 0.747419
Minibatch accuracy: 83.6%
Validation accuracy: 83.5%
Minibatch loss at step 3000: 0.802353
Minibatch accuracy: 82.0%
Validation accuracy: 83.5%
Test accuracy: 89.8%


In [25]:
### to verify overfitting by reducing the number of samples
### adding dropout layer with very low number of sample data

num_steps = 3001
trainData2 = trainData[:500,:]
trainLabels2 = trainLabels[:500]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (trainLabels2.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = trainData2[offset:(offset + batch_size), :]
    batch_labels = trainLabels2[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), validLabels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3617.600342
Minibatch accuracy: 11.7%
Validation accuracy: 27.9%
Minibatch loss at step 500: 21.182621
Minibatch accuracy: 100.0%
Validation accuracy: 77.9%
Minibatch loss at step 1000: 0.538705
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 1500: 0.336815
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 2000: 0.321005
Minibatch accuracy: 99.2%
Validation accuracy: 78.3%
Minibatch loss at step 2500: 0.295822
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 3000: 0.283438
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Test accuracy: 85.2%


In [26]:
import math as math

In [29]:
batch_size = 128
beta = 0.001

hidden_nodes_1 = 1024
hidden_nodes_2 = int(hidden_nodes_1 * 0.5)
hidden_nodes_3 = int(hidden_nodes_1 * np.power(0.5, 2))
hidden_nodes_4 = int(hidden_nodes_1 * np.power(0.5, 3))
hidden_nodes_5 = int(hidden_nodes_1 * np.power(0.5, 4))

graph = tf.Graph()
with graph.as_default():

    '''Input Data'''
    # For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(validData)
    tf_test_dataset = tf.constant(test_dataset)

    '''Variables'''
    # Hidden RELU layer 1
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes_1], stddev=math.sqrt(2.0/(image_size*image_size))))
    biases_1 = tf.Variable(tf.zeros([hidden_nodes_1]))

    # Hidden RELU layer 2
    weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes_1, hidden_nodes_2], stddev=math.sqrt(2.0/hidden_nodes_1)))
    biases_2 = tf.Variable(tf.zeros([hidden_nodes_2]))
    
    # Hidden RELU layer 3
    weights_3 = tf.Variable(tf.truncated_normal([hidden_nodes_2, hidden_nodes_3], stddev=math.sqrt(2.0/hidden_nodes_2)))
    biases_3 = tf.Variable(tf.zeros([hidden_nodes_3]))
    
    # Hidden RELU layer 4
    weights_4 = tf.Variable(tf.truncated_normal([hidden_nodes_3, hidden_nodes_4], stddev=math.sqrt(2.0/hidden_nodes_3)))
    biases_4 = tf.Variable(tf.zeros([hidden_nodes_4]))
    
    # Hidden RELU layer 5
    weights_5 = tf.Variable(tf.truncated_normal([hidden_nodes_4, hidden_nodes_5], stddev=math.sqrt(2.0/hidden_nodes_4)))
    biases_5 = tf.Variable(tf.zeros([hidden_nodes_5]))
    
    # Output layer
    weights_6 = tf.Variable(tf.truncated_normal([hidden_nodes_5, num_labels], stddev=math.sqrt(2.0/hidden_nodes_5)))
    biases_6 = tf.Variable(tf.zeros([num_labels]))
    
    '''Training computation'''
    
    # Hidden RELU layer 1
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    hidden_layer_1 = tf.nn.relu(logits_1)
    # Dropout on hidden layer: RELU layer
    keep_prob = tf.placeholder("float")
    hidden_layer_1_dropout = tf.nn.dropout(hidden_layer_1, keep_prob)
    
    
    # Hidden RELU layer 2
    logits_2 = tf.matmul(hidden_layer_1_dropout, weights_2) + biases_2
    hidden_layer_2 = tf.nn.relu(logits_2)
    # Dropout on hidden layer: RELU layer
    hidden_layer_2_dropout = tf.nn.dropout(hidden_layer_2, keep_prob)
    
    # Hidden RELU layer 3
    logits_3 = tf.matmul(hidden_layer_2_dropout, weights_3) + biases_3
    hidden_layer_3 = tf.nn.relu(logits_3)
    # Dropout on hidden layer: RELU layer
    hidden_layer_3_dropout = tf.nn.dropout(hidden_layer_3, keep_prob)
    
    # Hidden RELU layer 4
    logits_4 = tf.matmul(hidden_layer_3_dropout, weights_4) + biases_4
    hidden_layer_4 = tf.nn.relu(logits_4)
    # Dropout on hidden layer: RELU layer
    hidden_layer_4_dropout = tf.nn.dropout(hidden_layer_4, keep_prob)
    
    # Hidden RELU layer 5
    logits_5 = tf.matmul(hidden_layer_4_dropout, weights_5) + biases_5
    hidden_layer_5 = tf.nn.relu(logits_5)
    # Dropout on hidden layer: RELU layer
    hidden_layer_5_dropout = tf.nn.dropout(hidden_layer_5, keep_prob)
    
    # Output layer
    logits_6 = tf.matmul(hidden_layer_5_dropout, weights_6) + biases_6 
    
    # Normal loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_6, labels = tf_train_labels))
    # Loss function with L2 Regularization with decaying learning rate beta=0.5
    regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + \
                   tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(weights_4) + \
                   tf.nn.l2_loss(weights_5) + tf.nn.l2_loss(weights_6)
    loss = tf.reduce_mean(loss + beta * regularizers)

    '''Optimizer'''
    # Decaying learning rate
    global_step = tf.Variable(0)  # count the number of steps taken.
    start_learning_rate = 0.5
    learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 100000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training
    train_prediction = tf.nn.softmax(logits_6)
    
    # Predictions for validation 
    valid_logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    valid_relu_1 = tf.nn.relu(valid_logits_1)
    
    valid_logits_2 = tf.matmul(valid_relu_1, weights_2) + biases_2
    valid_relu_2 = tf.nn.relu(valid_logits_2)
    
    valid_logits_3 = tf.matmul(valid_relu_2, weights_3) + biases_3
    valid_relu_3 = tf.nn.relu(valid_logits_3)
    
    valid_logits_4 = tf.matmul(valid_relu_3, weights_4) + biases_4
    valid_relu_4 = tf.nn.relu(valid_logits_4)
    
    valid_logits_5 = tf.matmul(valid_relu_4, weights_5) + biases_5
    valid_relu_5 = tf.nn.relu(valid_logits_5)
    
    valid_logits_6 = tf.matmul(valid_relu_5, weights_6) + biases_6
    
    valid_prediction = tf.nn.softmax(valid_logits_6)
    
    # Predictions for test
    test_logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    test_relu_1 = tf.nn.relu(test_logits_1)
    
    test_logits_2 = tf.matmul(test_relu_1, weights_2) + biases_2
    test_relu_2 = tf.nn.relu(test_logits_2)
    
    test_logits_3 = tf.matmul(test_relu_2, weights_3) + biases_3
    test_relu_3 = tf.nn.relu(test_logits_3)
    
    test_logits_4 = tf.matmul(test_relu_3, weights_4) + biases_4
    test_relu_4 = tf.nn.relu(test_logits_4)
    
    test_logits_5 = tf.matmul(test_relu_4, weights_5) + biases_5
    test_relu_5 = tf.nn.relu(test_logits_5)
    
    test_logits_6 = tf.matmul(test_relu_5, weights_6) + biases_6
    
    test_prediction = tf.nn.softmax(test_logits_6)

In [30]:
num_steps = 15000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (trainLabels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = trainData[offset:(offset + batch_size), :]
    batch_labels = trainLabels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), validLabels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.110243
Minibatch accuracy: 9.4%
Validation accuracy: 10.3%
Minibatch loss at step 500: 1.826337
Minibatch accuracy: 78.9%
Validation accuracy: 82.9%
Minibatch loss at step 1000: 1.419180
Minibatch accuracy: 81.2%
Validation accuracy: 84.2%
Minibatch loss at step 1500: 1.217085
Minibatch accuracy: 83.6%
Validation accuracy: 83.5%
Minibatch loss at step 2000: 0.930344
Minibatch accuracy: 87.5%
Validation accuracy: 84.8%
Minibatch loss at step 2500: 0.951522
Minibatch accuracy: 85.9%
Validation accuracy: 85.5%
Minibatch loss at step 3000: 0.865924
Minibatch accuracy: 86.7%
Validation accuracy: 86.1%
Minibatch loss at step 3500: 0.784855
Minibatch accuracy: 87.5%
Validation accuracy: 86.0%
Minibatch loss at step 4000: 0.900906
Minibatch accuracy: 84.4%
Validation accuracy: 85.8%
Minibatch loss at step 4500: 0.808330
Minibatch accuracy: 85.2%
Validation accuracy: 86.4%
Minibatch loss at step 5000: 0.759036
Minibatch accuracy: 84.4%
Validation accuracy